In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\user_installedapps.csv")
data.head()

,userID,appID
0,1,357
1,1,360
2,1,362
3,1,365
4,1,375


# 摸数据

In [ ]:
temp = data.sort_values(by='appID')

In [ ]:
data[data['appID']==360]

In [ ]:
temp.sort_values(by='userID', axis=0,ascending=False)
temp.head()

In [ ]:
temp[temp['userID']>10].count()

# APP重合的方案（目前数据不足以应用）
# 1. 生成文件-每个app的用户规模

In [ ]:
out = data.groupby(['appID']).count()

In [ ]:
out.rename(columns=({'userID':'scale'}), inplace=True)
path = 'C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userScale.csv'
out.to_csv(path)
read = pd.read_csv(path)

In [ ]:
read.set_index('appID').head()

# 2. 生成文件-每个用户的APP数量

In [ ]:
out = data.groupby(['userID']).count()

In [ ]:
out.rename(columns=({'appID':'scale'}), inplace=True)
path = 'C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_appScale.csv'
out.to_csv(path)
read = pd.read_csv(path)

In [ ]:
read.set_index('userID').head()

# 3.1 生成文件-每个app所应对的userID（以数组的形式存储）

In [ ]:
def getUserList(df):
    return list(df['userID'])
out = data.groupby('appID').apply(getUserList)

In [ ]:
out.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_users.csv')

# 3.2 生成文件-每个userID所对应的app（以数组的形式存储）

In [2]:
def getAppList(df):
    return list(df['appID'])
out = data.groupby('userID').apply(getAppList)

In [3]:
out.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv')

# 4. 生成文件-（所有）APP的用户APP数量分布

In [ ]:
readAPP = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userScale.csv')
readUser = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_appScale.csv').set_index('userID')
readAppUser = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_users.csv', names=['users'])

In [ ]:
readAPP.head()

In [ ]:
readUser.head()

In [ ]:
readAppUser.head()

In [ ]:
# 统计分布，并返回string格式
## 计算分布的dict
dict = {}
def setDict(users):
    for i in users:
        scale = readUser.at[int(i), 'scale']
        if dict.has_key(scale):
            dict[scale] = dict[scale] + 1
        else:
            dict[scale] = 1
## 统计分布
def getDist(appID):
    global dict
    dict = {} #先把dict置为空
    users = readAppUser.at[int(appID),'users'][1:-1].split(',')
    setDict(users)
    return dict

In [ ]:
readAPP['distribution'] = readAPP['appID'].apply(getDist)

In [ ]:
readAPP.head()

In [ ]:
readAPP[['appID', 'distribution']].to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userDistribution.csv',index=False)

# 5. 生成文件-（所有app→50）APP期望共现

In [ ]:
# 获得训练样本中APP及其Scale
dataTrain = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
dataTrain = pd.merge(dataTrain, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv'), on='creativeID')
readAppScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userScale.csv').set_index('appID')
dataTrain = dataTrain[['appID','label']].groupby(['appID']).count()
dataTrain.reset_index(inplace = True)

In [ ]:
dataTrain = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')

In [ ]:
dataTrain['scale'] = dataTrain['appID'].apply(lambda x: readAppScale.at[x,'scale'])

In [ ]:
dataTrain['appID'].values

In [ ]:
readAPP['appID'].values

In [ ]:
np.intersect1d(dataTrain['appID'].values, readAPP['appID'].values)

In [ ]:
readApp_userDistribution = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userDistribution.csv').set_index('appID')

In [ ]:
dict = eval(readApp_userDistribution.at[354,'distribution'])

In [ ]:
dict

# 4. 生成文件-（所有app→50）APP实际共现

# 5. 生成文件-（用户→50）正票数

# 6. 生成文件-（用户→50）负票数